# Sprawozdanie laboratorium pierwsze
## Reversi
Witold Jagiełło-Królikowski s21165

Adrian Bloch s21488


In [2]:
import numpy as np
from easyAI import TwoPlayerGame

In [3]:
to_string = lambda a: "ABCDEFGH"[a[0]] + str(a[1] + 1)
to_array = lambda s: np.array(["ABCDEFGH".index(s[0]), int(s[1]) - 1])

### inicjujemy zmienne

## Tworzymy klase Reversi i przekazujemy do niej TwoPlayerGame z biblioteki easyAI


In [4]:

class Reversi(TwoPlayerGame):

    def __init__(self, players, board=None):
        self.players = players
        self.board = np.zeros((8, 8), dtype=int)
        self.board[3, [3, 4]] = [1, 2]
        self.board[4, [3, 4]] = [2, 1]
        self.current_player = 1
#tworzymy nowa plansze

    def possible_moves(self):
        #jede mozliwe ruchy ktore mozemy wykonac to takie ktore sprawia, ze doprowadzimy do pokazania reversu przeciwnika ;)
        return [
            to_string((i, j))
            for i in range(8)
            for j in range(8)
            if (self.board[i, j] == 0)
               and (pieces_flipped(self.board, (i, j), self.current_player) != []) #wykonujemy tylko kiedy możemy wykonac ruch
        ]

    def make_move(self, pos):
        #kladziemy pionek koloru gracza na wyznaczonej pozycji i zmieniamy kolor pionkow ktore owy ruch zmieni
        pos = to_array(pos)
        flipped = pieces_flipped(self.board, pos, self.current_player)
        for i, j in flipped:
            self.board[i, j] = self.current_player
        self.board[pos[0], pos[1]] = self.current_player

    def show(self):
        # pokazujemy plansze w czytelnym dla ludzkiego oka formacie
        print(
            "\n"
            + "\n".join(
                ["  1 2 3 4 5 6 7 8"]
                + [
                    "ABCDEFGH"[k]
                    + " "
                    + " ".join(
                        [[".", "1", "2", "X"][self.board[k][i]] for i in range(8)]
                    )
                    for k in range(8)
                ]
                + [""]
            )
        )

    def is_over(self):
        #zalozyliśmy, że gra kończy się w momencie gdy któryś z graczy nie może wykonac ruchu, na potrzeby zaprezentowania działania stwierdziliśmy, że to wystarczy na ukazanie przebiegu gry
        return self.possible_moves() == []

    def scoring(self):
        #Na poczatkowym etapie gry więcej daje nam zajęcie miejsc przy granicach mapy z racji na brak możliwości zmiany ich koloru przez przeciwnika, po przekroczeniu 32 pionków na planszy (mniej więcej połowa gry) bardziej liczy się już ilość pionków niż sama ich pozycja,


        if np.sum(self.board == 0) > 32:  # nie ma jeszcze połowy wypełnionej
            player = (self.board == self.current_player).astype(int)
            opponent = (self.board == self.opponent_index).astype(int)
            return ((player - opponent) * BOARD_SCORE).sum() #oceniamy pkt na podstawie wybranych przez nich pul
        else:
            npieces_player = np.sum(self.board == self.current_player)
            npieces_opponent = np.sum(self.board == self.opponent_index)
            return npieces_player - npieces_opponent #oceniamy na podstawie ilości pionkow


## Tak wygląda storzona przez nasz plansza którą przekazujemy AI aby bardziej faworyzowała krańce mapy jako ruch.

In [5]:
BOARD_SCORE = np.array(
    [
        [9, 3, 3, 3, 3, 3, 3, 9],
        [3, 1, 1, 1, 1, 1, 1, 3],
        [3, 1, 1, 1, 1, 1, 1, 3],
        [3, 1, 1, 1, 1, 1, 1, 3],
        [3, 1, 1, 1, 1, 1, 1, 3],
        [3, 1, 1, 1, 1, 1, 1, 3],
        [3, 1, 1, 1, 1, 1, 1, 3],
        [9, 3, 3, 3, 3, 3, 3, 9],
    ]
)

DIRECTIONS = [
    np.array([i, j]) for i in [-1, 0, 1] for j in [-1, 0, 1] if (i != 0 or j != 0)
]

In [6]:

def pieces_flipped(board, pos, current_player):
    #Zwraca liste pozycji które powinny się zamienić jeżeli gracz połozy na danym miejscu 'pos' pionek

    flipped = []

    for d in DIRECTIONS:
        ppos = pos + d
        streak = [] # zerujemy liczba serii do zmiany
        while (0 <= ppos[0] <= 7) and (0 <= ppos[1] <= 7):
            if board[ppos[0], ppos[1]] == 3 - current_player:
                streak.append(+ppos)
            elif board[ppos[0], ppos[1]] == current_player:
                flipped += streak
                break
            else:
                break
            ppos += d
#
    return flipped


In [7]:

if __name__ == "__main__":
    from easyAI import AI_Player, AI_Player, Negamax

    # An example: Computer vs Computer:
    game = Reversi([AI_Player(Negamax(4)), AI_Player(Negamax(4))])

    game.play()
    if game.scoring() > 0:
        print("player %d wins." % game.current_player)
    elif game.scoring() < 0:
        print("player %d wins." % game.opponent_index)
    else:
        print("Draw."
              )#%%


  1 2 3 4 5 6 7 8
A . . . . . . . .
B . . . . . . . .
C . . . . . . . .
D . . . 1 2 . . .
E . . . 2 1 . . .
F . . . . . . . .
G . . . . . . . .
H . . . . . . . .


Move #1: player 1 plays C5 :

  1 2 3 4 5 6 7 8
A . . . . . . . .
B . . . . . . . .
C . . . . 1 . . .
D . . . 1 1 . . .
E . . . 2 1 . . .
F . . . . . . . .
G . . . . . . . .
H . . . . . . . .


Move #2: player 2 plays E6 :

  1 2 3 4 5 6 7 8
A . . . . . . . .
B . . . . . . . .
C . . . . 1 . . .
D . . . 1 1 . . .
E . . . 2 2 2 . .
F . . . . . . . .
G . . . . . . . .
H . . . . . . . .


Move #3: player 1 plays F3 :

  1 2 3 4 5 6 7 8
A . . . . . . . .
B . . . . . . . .
C . . . . 1 . . .
D . . . 1 1 . . .
E . . . 1 2 2 . .
F . . 1 . . . . .
G . . . . . . . .
H . . . . . . . .


Move #4: player 2 plays B5 :

  1 2 3 4 5 6 7 8
A . . . . . . . .
B . . . . 2 . . .
C . . . . 2 . . .
D . . . 1 2 . . .
E . . . 1 2 2 . .
F . . 1 . . . . .
G . . . . . . . .
H . . . . . . . .


Move #5: player 1 plays C6 :

  1 2 3 4 5 6 7 8
A . . . . .

KeyboardInterrupt: 